# Classification problem

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

## Dataset preparation

In [17]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import stats module from scipy, which contains a large number of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

# need to ignore the warnings
import warnings

### Data collection

In [18]:
# Path to the local dataset file (YOURS MAY BE DIFFERENT!)
DATA_PATH = './data/loan-prediction/dataset.csv'

# Load the dataset with Pandas
data = pd.read_csv(DATA_PATH, sep=',', index_col='Loan_ID')

# show result
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Handling missing values

The first thing we might do is to replace the NA values with the mean of all the values (in the case of numerical values). The reality is that with the presence of _outliers_, the mean might not be the best choice. The __median__ is a better solution, being indeed robust to the outliers in the dataset.

In [19]:
from pandas.api.types import is_numeric_dtype

# removed NA values
data = data.apply(lambda x:
                  x.fillna(x.median()) if is_numeric_dtype(x)
                  else x.fillna(x.mode().iloc[0]))

# show result
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,145.752443,342.410423,0.855049
std,6109.041673,2926.248369,84.107233,64.428629,0.352339
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2877.500000,0.000000,100.250000,360.000000,1.000000
50%,3812.500000,1188.500000,128.000000,360.000000,1.000000
75%,5795.000000,2297.250000,164.750000,360.000000,1.000000
max,81000.000000,41667.000000,700.000000,480.000000,1.000000


### Encoding categorical features - _One-hot Encoding_

Categorical values should be transformed into numerical values to be used in the machine-learning pipeline. Not all the ML models can support categorical values.

This procedure is achieved by the <tt>get_dummies</tt> function.


In [20]:
# get categorical features
# not calculating Loan_Status beacuse it is binary but it is not numerical
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']

# get dummy function
data_with_dummies = pd.get_dummies(data, columns=categorical_features)

# check result
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True
LP001003,4583,1508.0,128.0,360.0,1.0,N,False,True,False,True,...,True,False,False,True,False,True,False,True,False,False
LP001005,3000,0.0,66.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,True,False,False,True,False,False,True
LP001006,2583,2358.0,120.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,False,True,True,False,False,False,True
LP001008,6000,0.0,141.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True


Move the predicted column to the last

In [21]:
# move predicted column to last
columns = data_with_dummies.columns.tolist()
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))
data_with_dummies = data_with_dummies.loc[:, columns]

# check result
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,Y
LP001003,4583,1508.0,128.0,360.0,1.0,False,True,False,True,False,...,False,False,True,False,True,False,True,False,False,N
LP001005,3000,0.0,66.0,360.0,1.0,False,True,False,True,True,...,False,False,True,False,False,True,False,False,True,Y
LP001006,2583,2358.0,120.0,360.0,1.0,False,True,False,True,True,...,False,False,False,True,True,False,False,False,True,Y
LP001008,6000,0.0,141.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,Y


### Encoding binary class label

To make the binary class labels in a numerical value, first identify the col and the two possible values. Then replace the with 1 and -1.

In [22]:
# replace data with dummies
data = data_with_dummies

# replace binary labels with binary numerical values
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x == 'Y' else -1)

# check result
data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,1
LP001003,4583,1508.0,128.0,360.0,1.0,False,True,False,True,False,...,False,False,True,False,True,False,True,False,False,-1
LP001005,3000,0.0,66.0,360.0,1.0,False,True,False,True,True,...,False,False,True,False,False,True,False,False,True,1
LP001006,2583,2358.0,120.0,360.0,1.0,False,True,False,True,True,...,False,False,False,True,True,False,False,False,True,1
LP001008,6000,0.0,141.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,1


## Build the model

In [23]:
from sklearn.metrics            import get_scorer
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn                    import tree

# cross validation
from sklearn.model_selection    import KFold
from sklearn.model_selection    import StratifiedKFold
from sklearn.model_selection    import cross_val_score
from sklearn.model_selection    import cross_validate
from sklearn.model_selection    import train_test_split

# hyperparams optimization
from sklearn.model_selection    import GridSearchCV
from sklearn.metrics            import accuracy_score
from sklearn.metrics            import roc_auc_score
from sklearn.metrics            import f1_score
from sklearn.metrics            import precision_score
from sklearn.metrics            import classification_report
from sklearn.metrics            import explained_variance_score

# models
from sklearn.linear_model       import LogisticRegression
from sklearn.svm                import LinearSVC
from sklearn.svm                import SVC
from sklearn.tree               import DecisionTreeClassifier
from sklearn.tree               import DecisionTreeRegressor
from sklearn.neighbors          import KNeighborsRegressor
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.ensemble           import RandomForestClassifier
from sklearn.ensemble           import AdaBoostClassifier
from sklearn.ensemble           import GradientBoostingClassifier
from sklearn.ensemble           import RandomForestRegressor

#from sklearn.externals import joblib

### Split the dataset

In [24]:
# extract dataset X from the DataFrame
X = data.iloc[:, : -1]
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,False,True,True,False,True,False,False,False,True,False,True,False,False,False,True
LP001003,4583,1508.0,128.0,360.0,1.0,False,True,False,True,False,True,False,False,True,False,True,False,True,False,False
LP001005,3000,0.0,66.0,360.0,1.0,False,True,False,True,True,False,False,False,True,False,False,True,False,False,True
LP001006,2583,2358.0,120.0,360.0,1.0,False,True,False,True,True,False,False,False,False,True,True,False,False,False,True
LP001008,6000,0.0,141.0,360.0,1.0,False,True,True,False,True,False,False,False,True,False,True,False,False,False,True


In [25]:
# extract the target
y = data.iloc[:, -1]
y.head()

Loan_ID
LP001002    1
LP001003   -1
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

Let's split our dataset with __scikit-learn__ <tt>train_test_split</tt> function, which splits the input dataset into a training set and a test set, respectively.

We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.

Additionally, we would like to take advantage of _stratified_ sampling to obtain the same target distribution in both the training and the test sets.


In [26]:
# fixed random state
RND_SEED = 653

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=RND_SEED,
                                                    shuffle=True)

### Evaluate function

We can create a function such that it will print the evaluation of the prediction.

In [27]:
"""
General function used to assess the quality of predictions in terms of two scores:
 - accuracy 
 - ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
        
        print("Precision")
        print(f"   {precision_score(true_values, predicted_values) }")
        
        print("\nF1-score")
        print(f"   {f1_score(true_values, predicted_values) }")

### Stratified k-fold cross-validation

An even better option is to use a stratified k-fold validation. This variant splits the dataset in a way such that every fold contains the same proportion of features.

In [28]:
# define the model
model = RandomForestClassifier()

# define stratified k-fold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# perform the cross-validation
cross_validation = cross_validate(model, X, y,
                                  cv = folds,
                                  scoring=('precision', 'f1'),
                                  return_train_score=True)

# print result
pd.DataFrame(cross_validation)

,fit_time,score_time,test_precision,train_precision,test_f1,train_f1
0,0.289243,0.015957,0.816327,1.0,0.869565,1.0
1,0.265315,0.012965,0.816327,1.0,0.869565,1.0
2,0.210437,0.012941,0.800000,1.0,0.869565,1.0
3,0.212433,0.014958,0.764706,1.0,0.838710,1.0
4,0.219437,0.014937,0.822222,1.0,0.850575,1.0
5,0.207442,0.012965,0.750000,1.0,0.829787,1.0
6,0.201461,0.012965,0.826087,1.0,0.863636,1.0
7,0.212408,0.012966,0.730769,1.0,0.808511,1.0
8,0.207421,0.013962,0.812500,1.0,0.866667,1.0
9,0.204429,0.012989,0.847826,1.0,0.886364,1.0


In [29]:
# print averages
print("Mean of the TEST SET scores")
print(f"Precision: {np.mean(cross_validation['test_precision']): .3f}")
print(f"F1-score : {np.mean(cross_validation['test_f1']): .3f}")

Mean of the TEST SET scores
Precision:  0.799
F1-score :  0.855


## Comparing different models

There might be a situation where different models can be compared to see which one fits better to the classification problem we need to solve.

### Select the best hyper-params of a fixed family of model

In this first case, we study the influence different hyper-params have on the same family model (logistic regression) and choose the best

In [35]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=RND_SEED)

# define folds
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=True)

# get the model
model = RandomForestClassifier()

# get dictionary of hyperparameters
hyperparams = {
    'n_estimators' : [5, 10, 25, 50],
    'max_depth' : [1, 5, 10, 25, 50]
}

# use Grid Search to compare all the combination
grid_search = GridSearchCV(model, hyperparams,
                           cv = folds,
                           scoring = ('f1'),
                           return_train_score=True,
                           verbose=True)

# find the solution
grid_search.fit(X_train, y_train)

# display result
pd.DataFrame(grid_search.cv_results_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.047174,0.021384,0.023138,0.019790,1,5,"{'max_depth': 1, 'n_estimators': 5}",0.809524,0.819277,0.819277,...,0.818182,0.880117,0.818182,0.818182,0.818182,0.878477,0.880117,0.818182,0.836817,0.027995
1,0.021543,0.005392,0.006582,0.001493,1,10,"{'max_depth': 1, 'n_estimators': 10}",0.809524,0.819277,0.819277,...,0.818182,0.880117,0.818182,0.818182,0.819277,0.818182,0.850767,0.849162,0.832398,0.020120
2,0.050264,0.010698,0.007682,0.002637,1,25,"{'max_depth': 1, 'n_estimators': 25}",0.809524,0.819277,0.819277,...,0.819277,0.818182,0.836763,0.818182,0.818182,0.820375,0.818182,0.820375,0.822377,0.007071
3,0.080485,0.012996,0.007482,0.001679,1,50,"{'max_depth': 1, 'n_estimators': 50}",0.809524,0.819277,0.819277,...,0.818182,0.821477,0.822581,0.844875,0.821477,0.818182,0.818182,0.840659,0.824307,0.009407
4,0.014268,0.003435,0.006179,0.001529,5,5,"{'max_depth': 5, 'n_estimators': 5}",0.846154,0.871795,0.868421,...,0.886931,0.890208,0.889543,0.883309,0.888563,0.887240,0.886263,0.899851,0.890348,0.004917
5,0.027022,0.003297,0.006388,0.001557,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.883117,0.871795,0.906667,...,0.879536,0.897059,0.892805,0.890511,0.885965,0.887591,0.876990,0.890511,0.888825,0.006255
6,0.047580,0.004043,0.006480,0.001491,5,25,"{'max_depth': 5, 'n_estimators': 25}",0.883117,0.871795,0.906667,...,0.882438,0.885630,0.887591,0.888244,0.887591,0.887591,0.891496,0.889217,0.886683,0.002722
7,0.085273,0.004568,0.007378,0.001344,5,50,"{'max_depth': 5, 'n_estimators': 50}",0.883117,0.871795,0.906667,...,0.881752,0.883041,0.894428,0.887918,0.888889,0.885630,0.888889,0.883382,0.886684,0.003798
8,0.015555,0.003772,0.005886,0.001750,10,5,"{'max_depth': 10, 'n_estimators': 5}",0.805556,0.821918,0.876712,...,0.929448,0.958991,0.940252,0.937020,0.926380,0.937304,0.938776,0.940994,0.941647,0.010162
9,0.028128,0.003562,0.006681,0.001264,10,10,"{'max_depth': 10, 'n_estimators': 10}",0.826667,0.871795,0.918919,...,0.938650,0.953271,0.948357,0.961783,0.957614,0.954617,0.957480,0.948518,0.951421,0.006494


In [36]:
# get best combination
print(f"Best hyperparameter:")
print(grid_search.best_params_)
print(f"Best F1-score: {grid_search.best_score_:.3}")

Best hyperparameter:
{'max_depth': 5, 'n_estimators': 25}
Best F1-score: 0.878


In [38]:
# define model with best hyperparams
model = RandomForestClassifier(max_depth=grid_search.best_params_['max_depth'], n_estimators=grid_search.best_params_['n_estimators'])

# train model on whole dataset
model.fit(X_train, y_train)

# evaluate the prediction capabilities
evaluate(y_test, model.predict(X_test))

Precision
   0.7714285714285715

F1-score
   0.8663101604278075


### Best model from fixed hyper-params

Here we fix the hyper-params for each model (we use the default params) and compare the different models

In [39]:
# ignore warnings
warnings.filterwarnings('ignore')

# split dataset

# define models
models = {
    'LogisticRegression'        : LogisticRegression(),
    'KNeighborsClassifier'      : KNeighborsClassifier(),
    'DecisionTreeClassifier'    : DecisionTreeClassifier(),
    'RandomForestClassifier'    : RandomForestClassifier()
}

# define folds
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# cross validate the models manually
cross_validation_scores = dict()
for model_name, model in models.items():
    cross_validation_scores[model_name] = cross_val_score(model, X_train, y_train,
                                                          cv=folds,
                                                          scoring=('f1'))


# save results in a DataFrame
cross_validation_scores = pd.DataFrame(cross_validation_scores).transpose()

# compute mean and std-dev
cross_validation_scores['mean'] = np.mean(cross_validation_scores, axis=1)
cross_validation_scores['std'] = np.std(cross_validation_scores, axis=1)
cross_validation_scores = cross_validation_scores.sort_values(['mean', 'std'], ascending=False)

# print result
cross_validation_scores

,0,1,2,3,4,5,6,7,8,9,mean,std
LogisticRegression,0.840580,0.864865,0.871795,0.906667,0.906667,0.850000,0.883117,0.894737,0.860759,0.861111,0.874030,0.020824
RandomForestClassifier,0.823529,0.845070,0.826667,0.880000,0.918919,0.860759,0.868421,0.837838,0.842105,0.840580,0.854389,0.026078
KNeighborsClassifier,0.729730,0.804878,0.736842,0.829268,0.746667,0.739726,0.784810,0.800000,0.732394,0.783784,0.768810,0.032503
DecisionTreeClassifier,0.741935,0.746269,0.771429,0.828571,0.800000,0.676923,0.764706,0.753623,0.800000,0.746269,0.762972,0.037674


In [43]:
# save the best model
print(f"{cross_validation_scores.index[0]}\n")
model = models[cross_validation_scores.index[0]]

# re-train the best model on the whole train set
model.fit(X_train, y_train)

# evaluate the test set predicion
evaluate(y_test, model.predict(X_test))

LogisticRegression

Precision
   0.7722772277227723

F1-score
   0.8524590163934426
